# Descripción
   ### Los datos están relacionados con campañas de marketing directo de una institución bancaria. Las campañas de marketing se basaron en llamadas telefónicas. A menudo, se requería más de un contacto con el mismo cliente, para poder acceder si el producto (depósito a plazo bancario) estaría ('sí') o no ('no') suscrito.


    Reglas y método de evaluación
    
    El desafío consistirá en que cada grupo formado deberá entregar un arreglo con los resultados ('yes' o 'no') y se medirá la medida de desempeño F1 score (macro).
    
    EL archivo de entrega debe ser en formato csv sin indice. Debe ser una sola columna con valores ´yes´ o ´no´ en 5210 filas


#### Variables de entrada:

# datos del cliente bancario:
1 - edad (numérico)

2 - trabajo: tipo de trabajo (categórico: 'admin.', 'Obrero', 'emprendedor', 'empleada doméstica', 'gerencia', 'jubilado', 'autónomo', 'servicios', 'estudiante' , 'técnico', 'desempleado', 'desconocido')

3 - marital: estado civil (categórico: 'divorciado', 'casado', 'soltero', 'desconocido'; nota: 'divorciado' significa divorciado o viudo)

4 - educación (categórica: 'básico.4y', 'básico.6y', 'básico.9y', 'escuela secundaria', 'analfabeto', 'curso.profesional', 'título universitario', 'desconocido')

5 - incumplimiento: ¿tiene crédito en incumplimiento? (categórico: 'no', 'sí', 'desconocido')

6 - vivienda: ¿tiene préstamo para vivienda? (categórico: 'no', 'sí', 'desconocido')

7 - préstamo: ¿tiene préstamo personal? (categórico: 'no', 'sí', 'desconocido')

# relacionado con el último contacto de la campaña actual:

8 - contacto: tipo de comunicación de contacto (categórico: 'celular', 'teléfono')

9 - mes: último mes de contacto del año (categórico: 'jan', 'feb', 'mar', ..., 'nov', 'dec')

10 - day_of_week: último día de contacto de la semana (categórico: 'lun', 'tue', 'mié', 'jue', 'vie')

11 - duración: duración del último contacto, en segundos (numérico). Nota importante: este atributo afecta en gran medida el objetivo de salida (por ejemplo, si duración = 0, entonces y = 'no'). Sin embargo, se desconoce la duración antes de realizar una llamada. Además, después de la finalización de la llamada y, obviamente, se conoce. Por lo tanto, esta entrada solo debe incluirse con fines de referencia y debe descartarse si la intención es tener un modelo predictivo realista.

# otros atributos:

12 - campaña: número de contactos realizados durante esta campaña y para este cliente (numérico, incluye último contacto)

13 - pdays: número de días que pasaron después de que el cliente fue contactado por última vez desde una campaña anterior (numérico; 999 significa que el cliente no fue contactado previamente)

14 - anterior: número de contactos realizados antes de esta campaña y para este cliente (numérico)

15 - poutcome: resultado de la campaña de marketing anterior (categórico: 'fracaso', 'inexistente', 'éxito')

Variable de salida (objetivo deseado):

21 - y - ¿el cliente ha suscrito un depósito a plazo? (binario: 'sí', 'no')


# Comenzando

In [ ]:
import numpy as np
import pandas as pd
import pylab as plt
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing


from imblearn.over_sampling import RandomOverSampler
import imblearn

In [ ]:
df_train = pd.read_csv('Trainset.csv', index_col= 'Unnamed: 0')
df_test = pd.read_csv('TestFeatures.csv')
final = pd.read_csv('submission_example.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head()

## Eliminar Columnas
duration   



In [ ]:
df_train = df_train.drop(['duration'], axis=1)
df_train.head()

## Distribution

In [ ]:
# df_train.hist(figsize=(12,12), color = 'darkblue')
# plt.show()

### Mayormente los usuarios NO tiene contratado el producto

In [ ]:
df_train.y.value_counts()

In [ ]:
df_train.y.value_counts()/df_train.shape[0]

# Preproceso de datos Ecoding Test Train

### Copias de los datos.

In [ ]:
# Datos sin balanceo
df_prep = df_train.copy()

# Encoding de datos Train

## encoding datos

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()

df_prep['job']=labelencoder.fit_transform(df_prep['job'].values)
df_prep['marital']=labelencoder.fit_transform(df_prep['marital'].values)
df_prep['education']=labelencoder.fit_transform(df_prep['education'].values)
df_prep['contact']=labelencoder.fit_transform(df_prep['contact'].values)
df_prep['poutcome']=labelencoder.fit_transform(df_prep['poutcome'].values)

df_prep['default'].replace(['yes','no'],[1,0],inplace=True)
df_prep['housing'].replace(['yes','no'],[1,0],inplace=True)
df_prep['loan'].replace(['yes','no'],[1,0],inplace=True)
df_prep['y'].replace(['yes','no'],[1,0],inplace=True)
df_prep['month'].replace(['jan','feb','mar','apr','may','jun','jul','aug' , 'sep','oct','nov','dec'],
                         [1,2,3,4,5,6,7,8,9,10,11,12],inplace=True)



#CATEGORICAS ['job','marital','education','contact','month','poutcome']
#NUMERICAS ['age','balance','day','campaign','pdays','previous']
#BINARIAS  ['default','housing','loan']  

In [ ]:
df_prep.head()

In [ ]:
df_prep['y'].unique(),df_prep['y'].value_counts()

## Normalizacion Train

In [ ]:
# #selecionamos las columnas que normalizaremos
df_norm = df_prep[['age','balance','campaign','pdays','previous','poutcome']]
df_prep2 = df_prep.drop(columns=['age','balance','campaign','pdays','previous','poutcome'])

from sklearn.preprocessing import MinMaxScaler
df_trans = MinMaxScaler()
df_trans = df_trans.fit_transform(df_norm)

df_trans = pd.DataFrame(df_trans)
df_trans.columns = df_norm.columns

df_train_trans = pd.concat([df_trans, df_prep2], axis=1,)
df_train_trans.reindex(columns=['age','job','marital','education','default','balance','housing',
                          'loan','contact','day','month','campaign','pdays','previous','poutcome','y'])

df_train_trans.head()

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# train_trans  = MinMaxScaler()
# train_trans  = train_trans.fit_transform(df_prep)

# from sklearn.preprocessing import Normalizer
# test_trans  = Normalizer()
# test_trans  = test_trans.fit_transform(df_prep)


# #crearmamos el dataset
# train_trans = pd.DataFrame(train_trans)
# train_trans.columns = df_prep.columns
# train_trans.reindex(columns=['age','job','marital','education','default','balance','housing',
#                          'loan','contact','day','month','campaign','pdays','previous','poutcome','y'])


# Preproceso datos test

## encoding datos sin balanceo

#### Esto es necesario para poder probar el modelo sobre la misma estructura de datos pre_procesados de train.

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
df_test['job']=labelencoder.fit_transform(df_test['job'].values)
df_test['marital']=labelencoder.fit_transform(df_test['marital'].values)
df_test['education']=labelencoder.fit_transform(df_test['education'].values)
df_test['contact']=labelencoder.fit_transform(df_test['contact'].values)
df_test['poutcome']=labelencoder.fit_transform(df_test['poutcome'].values)

df_test['default'].replace(['yes','no'],[1,0],inplace=True)
df_test['housing'].replace(['yes','no'],[1,0],inplace=True)
df_test['loan'].replace(['yes','no'],[1,0],inplace=True)
df_test['month'].replace(['jan','feb','mar','apr','may','jun','jul','aug' , 'sep','oct','nov','dec'],
                         [1,2,3,4,5,6,7,8,9,10,11,12],inplace=True)

#CATEGORICAS ['job','marital','education','contact','month','poutcome']
#NUMERICAS ['age','balance','day','campaign','pdays','previous']
#BINARIAS  ['default','housing','loan']  

In [ ]:
test_trans = df_test

## Normalizacion Test

In [ ]:
df_norm_test = df_test[['age','balance','campaign','pdays','previous']]
df_prep_test = df_test.drop(columns=['age','balance','campaign','pdays','previous','duration'])
from sklearn.preprocessing import MinMaxScaler
df_trans = MinMaxScaler()
df_trans = df_trans.fit_transform(df_norm_test)
df_trans = pd.DataFrame(df_trans)
df_trans.columns = df_norm_test.columns
test_trans = pd.concat([df_trans, df_prep_test], axis=1,)
test_trans.reindex(columns=['age','job','marital','education','balance','housing',
                         'loan','contact','day','month','campaign','pdays','previous','poutcome'])

test_trans.head()

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# test_trans  = MinMaxScaler()
# test_trans  = test_trans.fit_transform(df_test)

# from sklearn.preprocessing import Normalizer
# test_trans  = Normalizer()
# test_trans  = test_trans.fit_transform(df_test)

In [ ]:
# #rearmamos el dataset
# test_trans = pd.DataFrame(test_trans)
# test_trans.columns = df_test.columns

# Balanceo de datos

# Oversampling

In [ ]:
#Separamos el set de datos
x = df_train_trans.drop(['y'], axis=1)
y = df_train_trans['y']

In [ ]:
random_over = RandomOverSampler(sampling_strategy='auto',
                               random_state=123)

X_over,y_over = random_over.fit_resample(x,y)
df_over = X_over
df_over['y'] = y_over

print(df_over.y.value_counts()/df_over.shape[0])
print(df_over.y.value_counts())
print(df_over.shape)
df_over.head()

# Undersampling

In [ ]:
from sklearn.utils import resample, shuffle

#set the minority class to a seperate dataframe
df_yes = df_train_trans[df_train['y'] == 'yes']
#set other classes to another dataframe
df_no = df_train_trans[df_train['y'] == 'no']  

#upsample the class
df_no_upsampled = resample(df_no,
                           random_state=123,
                           n_samples=len(df_yes),
                           replace=False)

#concatenate the upsampled dataframe
df_under = pd.concat([df_no_upsampled,df_yes])

print(df_under.y.value_counts()/df_under.shape[0])
print(df_under.y.value_counts())
print(df_under.shape)
df_under.head()

#### Tenemos dos set de datos Balanceados en dos formas diferentes

# Ejecutando Modelos 

In [ ]:
#df = df_train_trans
#df = df_over
#df = df_under
#df_prep

In [ ]:
#df_over = df_over.sample(frac=0.95, random_state=786)

In [ ]:
from pycaret.classification import *
clf1 = setup(data = df_prep,
             target = 'y',
             train_size=0.3,
             session_id=123,
             fold_shuffle=True,
             data_split_shuffle=False,
             normalize = True,
             normalize_method = robust
            
            )

In [ ]:
best_model = compare_models(sort = 'F1')

    Precision nos da la calidad de la predicción: ¿qué porcentaje de los que hemos dicho que son la clase positiva, en realidad lo son?
    
    Recall nos da la cantidad: ¿qué porcentaje de la clase positiva hemos sido capaces de identificar?
    
    F1 combina Precision y Recall en una sola medida
    
    La Matriz de Confusión indica qué tipos de errores se cometen

###### Evaluando F1 antes de continuar con los modelos

    modelo_1 = 

    modelo_2 = 

    modelo_3 = 0.3472 xgboost
    
    modelo_4 = 0.3478 qda
    
    modelo_5 = 0.3262 qda   
    
    modelo_6_over = 0.88 RF
    
    modelo_7_under = 0.77 gbc

In [ ]:
print(best_model)

In [ ]:
models()

In [ ]:
rf = create_model('catboost')

# Optimizando Hiperparametros
### por F1

In [ ]:
tuned_rf = tune_model(rf)

In [ ]:
print(tuned_rf)

# Ensamble

In [ ]:
#boosted_dt = ensemble_model (ctb, method = 'Boosting', n_estimators = 100)

# entrenar un clasificador de ensacado en dt
bagged_dt = ensemble_model ( dt, method = 'Embolsado' )
# entrenar un clasificador adaboost en dt con 100 estimadores
boosted_dt = ensemble_model ( dt, method = 'Boosting' , n_estimators = 100 )

# Prediccion

In [ ]:
# predict_model(boosted_dt);

In [ ]:
rf_final = finalize_model(rf)
#Parámetros finales del modelo Random Forest para su despliegue a producción
print(rf)

In [ ]:
predict_model(rf);

# Testeando resultados.

In [ ]:
test_predictions = predict_model(rf_final, data = test_trans)
test_predictions.head()

In [ ]:
test_predictions.info()

# Resultado

In [ ]:
from pycaret.utils import check_metric

check_metric(test_predictions['default'], test_predictions['Label'], metric = 'Accuracy')

accura_m1 =

accura_m2 = 0.9232 - puntaje 49.2757

    fix_imbalance = True, 
                 train_size = 0.2, 
                 iterative_imputation_iters = 5,
                 normalize = True,
                 normalize_method = 'minmax',
                 transformation = True,
                 high_cardinality_features = ['job','education','month'],
                 numeric_features = ['age','balance','day','campaign','pdays','previous'] )
             
accura_m3 = 0.9378  - puntaje 
     
     Es necesario preprocesar datos de test y train
      
accura_m3 = 0.9798
     
     Aplicamos modelo qda



accura_m3 = 0.9384 

    Aplicamos catboost
    
accura_m4_over = 0.7417

accura_m6_over_prep =0.9313




# Guardando datos para carga

In [ ]:
test_predictions['Label'].replace([1,0],['yes','no'],inplace=True)

In [ ]:
label = test_predictions[['Label']]

In [ ]:
label.to_csv('example_4.csv',index=False , header = False )